In [2]:
import time

import itertools

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import torch

In [3]:
start = time.time()
print("Loading Model")

# AIMA -> explicar no contexto de busca
obs = ['This room is called the foundry.']

context = ['you are in workshop', 'you are in foundry']
#

a = list(itertools.product(obs, context))
a.sort(key=lambda x: x[1])
print(f"Combinations = {len(a)}")

max_length = 256

hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
#hg_model_hub_name = "alisawuffles/roberta-large-wanli"
config = AutoConfig.from_pretrained(hg_model_hub_name)
entailment_idx = config.label2id['entailment']

config = AutoConfig.from_pretrained(hg_model_hub_name)
tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name)

print(config.label2id)

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)
model.to(device)

end = time.time()
print(f"Model loaded {end - start} - model {model.device}")

Loading Model
Combinations = 2
{'entailment': 0, 'neutral': 1, 'contradiction': 2}


Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded 6.353092193603516 - model cpu


In [4]:
start = time.time()
tokenized_input_seq_pair = tokenizer.batch_encode_plus(a,
                                                       max_length=max_length,
                                                       return_token_type_ids=True, truncation=True, padding=True)

input_ids = torch.tensor(tokenized_input_seq_pair['input_ids'], device=device).long()  #.unsqueeze(0)
# remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
token_type_ids = torch.tensor(tokenized_input_seq_pair['token_type_ids'], device=device).long()  #.unsqueeze(0)
attention_mask = torch.tensor(tokenized_input_seq_pair['attention_mask'], device=device).long()  #.unsqueeze(0)

outputs = model(input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=None)
end = time.time()

print(f"Inference time: {end - start}")
logits = outputs[0]
probs = torch.softmax(logits, dim=1)
for i, pair in enumerate(a):
    predicted_probability = probs[i].tolist()  # batch_size only one
    print('----')
    print("Premise:", pair[0])
    print("Hypothesis:", pair[1])
    print("Entailment:", predicted_probability[int(config.label2id['entailment'])])
    print("Neutral:", predicted_probability[int(config.label2id['neutral'])])
    print("Contradiction:", predicted_probability[int(config.label2id['contradiction'])])

Inference time: 0.28208255767822266
----
Premise: This room is called the foundry.
Hypothesis: you are in foundry
Entailment: 0.9029870629310608
Neutral: 0.09309663623571396
Contradiction: 0.003916335757821798
----
Premise: This room is called the foundry.
Hypothesis: you are in workshop
Entailment: 0.5612775683403015
Neutral: 0.3587671220302582
Contradiction: 0.07995528727769852


- $context = (c_1, c_2)$
- $beliefbase = (b_1, b_2)$
- $(b_1 \models c_1 \lor b_2 \models c_1) \land (b_1 \models c_2 \lor b_2 \models c_2)$

In [30]:
num_context = len(context)
num_beliefs = len(obs)

argmax_probs = probs.argmax(-1)
slice_idx = []
idx = 0
for i in range(num_context):
    slice_idx.append(argmax_probs[idx:(idx + num_beliefs)])
    idx = num_beliefs
slice_idx

[tensor([1, 2, 2, 2, 2, 2, 2])]

In [31]:
#c1_or = torch.where(c1 == entailment_idx, True, False)
#c2_or = torch.where(c2 == entailment_idx, True, False)

context_or = [torch.where(c == entailment_idx, True, False).any().unsqueeze(0) for c in slice_idx]
all_ors = torch.concatenate(context_or)  # contiguous
entailment = all_ors.all()
if entailment:
    print(entailment.item())
else:
    print("not entailment")

True


In [32]:
# True when a c_n is entailed by b_n
entailment_mask = torch.where(argmax_probs == entailment_idx, True, False)
# [B,c1:B,c2:...:B,cn]
slice_idx = []
idx = 0
for i in range(num_context):  # [c1, ..., cn]
    slice_idx.append(argmax_probs[idx:(idx + num_beliefs)])
    idx = num_beliefs

# True if ANY context comparation is ENTAILED by an belief in belief base (OR)
context_or = [torch.where(c == entailment_idx, True, False).any().unsqueeze(0) for c in slice_idx]
or_tensor = torch.concatenate(context_or)  # at least one belief should entail a context (OR)
and_result = or_tensor.all()  # all context must be entailed by the belief base (AND)
entailment_result = and_result.item()  # boolean result

entailment_probs = (probs[:, entailment_idx] * entailment_mask)  # retrieving only entailment predictions
entailment_score = entailment_probs[entailment_probs != 0].mean()  # mean of all entailment predictions
entailment_score

tensor(0.9644, grad_fn=<MeanBackward0>)

In [33]:
argmax_probs[argmax_probs == 2]

tensor([2, 2, 2, 2, 2, 2])

In [9]:
entailment_probs = (probs[:, entailment_idx] * entailment_mask)
entailment_probs[entailment_probs != 0].mean()

NameError: name 'entailment_mask' is not defined

In [72]:
type(entailment_probs)

torch.Tensor